> 运行环境为Python3.8

# 第2章 数值与字符串

## 2.1 数值

### 2.1.1 浮点精度问题

浮点精度问题，如果在Python中直接输入`0.1+0.2`会看到“奇景”

In [3]:
0.1 + 0.2

0.30000000000000004

正确的做法是使用Python提供的内置模块：decimal,并使用字符串。

In [9]:
from decimal import Decimal

# 注意是字符串
print(Decimal('0.1')+Decimal('0.2'))
# 此处仍然有问题
print(Decimal(0.1)+Decimal(0.2))

0.3
0.3000000000000000166533453694


### 2.1.2 编程建议

1. 对于数值比较，Python自带了两个边界数值，简化代码逻辑：`float("-inf") < 任意数 < float("inf")`

2. 在写代码时，一些比较复杂的数字，比如950400 ，不如写成 11\*24\*3600 ；不需要担心性能，Python在运行前会提前计算。我们可以使用内置的dis模块，查看汇编代码，验证此方法。

In [39]:
import dis

def add(x, y):
    return x + y + 11*24

dis.dis(add)

  4           0 LOAD_FAST                0 (x)
              2 LOAD_FAST                1 (y)
              4 BINARY_ADD
              6 LOAD_CONST               1 (264)
              8 BINARY_ADD
             10 RETURN_VALUE


## 2.2 字符串

判断字符串是否只包含数字

In [10]:
'1234'.isdigit()

True

str.partition(seq)功能按照‘seq’分割字符串，并返回三个成员的元祖，对于对分割结果有逻辑判定时，此方法更好用

In [38]:
'我的名字：rohan'.partition('：')

('我的名字', '：', 'rohan')

In [12]:
# 分割符换为英文冒号，导致分割结果 两处为空。
'我的名字：rohan'.partition(':')

('我的名字：rohan', '', '')

维护替换字符表，批量替换字符串,避免多次使用replace()。

In [35]:
ex_str = 'This is my style!呀呀呀'
table = ex_str.maketrans('is呀','no拉')
ex_str.translate(table)

'Thno no my otyle!拉拉拉'

### 2.2.1 Jinja2

之前笔者对于jinja2的使理解用仅限于前后端不分离的项目，但是对于非结构化的字符串，我们应该避免直接拼接。就像避免直接拼接SQL，而应该使用SQLAlchemy

In [41]:
from jinja2 import Template

_MOVIE_TMPL = """\
Welcome, {{username}}.
{%for name, score in movies %}
* {{name}}, Score: {{score}}
{%- endfor %}
"""

movies = [('杀死比尔','93'),('八恶人','95'),('奥特曼', '80')]

tmpl = Template(_MOVIE_TMPL)
str_tmpl = tmpl.render(username = 'Rohan', movies = movies)
print(str_tmpl)

Welcome, Rohan.

* 杀死比尔, Score: 93
* 八恶人, Score: 95
* 奥特曼, Score: 80


如果要去掉字符串中的缩紧，可以使用`textwrap`库的`dedent`

### 2.2.2 建议

1. 字符串`r`开头的方法，都是逆序操作;
2. 字符串拼接自Python2.2 以后可以直接使用`+=`操作
3. 性能测试，可以直接使用`timeit`模块


In [48]:
import timeit

def foo():
    add(1,2)

# 默认执行100万次
timeit.timeit(setup='from __main__ import foo',stmt = 'foo()')

0.12847758299994894

# 第3章 容器类型

## 3.1 列表

函数内部传递的list默认是一个变量指向了同一个变量，“+=”操作会使list这种可变类型的变量，就地修改。而str则会生成新的。

In [6]:
def add_list(input_list:list)->list:
    input_list += [11]
    return input_list

my_list = ['hello']
out_list = add_list(my_list)

['hello', 11]


## 3.2 具名元组

此处特别介绍一下具名元组的一些用法，可见使用这种下标会*增加可读性*。

In [12]:
from collections import namedtuple

Hero = namedtuple('Hero','name, health, mana, attack')

zeo = Hero(name='Zeo', health=200, mana=150, attack=33)
print(f"Zeo的血量是{zeo.health},魔法是{zeo[2]}")

Zeo的血量是200,魔法是150;Hero(name='Zeo', health=200, mana=150, attack=33)


对于一些，需求变动比较大的函数，我们可以用时namedtuple作为返回值。可以可以保证后期维护的时候，最小变动原则。而且具有类似静态语言的特性。

In [6]:
from typing import NamedTuple

class Address(NamedTuple):
    """地址信息结果"""
    country: str
    province: str
    city: str
        
def get_address(a, b, c):
    return Address(country=a, province=b, city= c)

addr = get_address("中国", "山东", "青岛")
addr.city

'青岛'

## 3.3 字典

我们经常会遇到访问字典，但不确定字典中的Key，是否是存在，不存在则会抛出KeyError异常，这种情况称为边界情况(edge case)。

下面我们介绍一下Python中，如何处理这种情况：

一般情况：

In [16]:
examples_dict ={"one":1}

if 'two' in examples_dict:
    two = examples_dict['two']
else:
    two = 0
    

Python 中会更推崇下面的写法，更简洁，执行效率也更高。

In [15]:
try:
    two = examples_dict['two']
except KeyError:
    two = 0

但是Python中，对于字典有专门的处理方法：
- dict.get(key, default) 
- dict.setdefault(key, default)

对于`setdefault`，如果key，已经存在，那么后面的default则无效，Python只返回key的值

In [19]:
# 此时字典中仍然没有这个Key
two = examples_dict.get('two', 2)
'two' in examples_dict

False

In [23]:
two = examples_dict.setdefault('two', 2)
'two' in examples_dict

True

### 3.3.1 字典的有序或无序

自Python3.7以后，字典的有序性正式作为语言规范的一部分。
> Python3.6在优化字典类型底层实现的同时，带来了一个副作用：“字典变得有序了”。

对于旧版本的Python，你可以使用`collections.OrderedDict`,它和新特性来说，还是有细微的不同：用“==”判断时，dict不会考虑顺序，但是OrderedDict 会判定顺序是否一致。


In [34]:
from collections import OrderedDict

d1 = {"1":1,"2":2}
d2 = {"2":2, "1":1}
print(d1 ==d2)

od1 = OrderedDict()
od1["1"]=1
od1["2"]=2
od2 = OrderedDict()
od2['2']=2
od2['1']=1
print(od1==od2)

True
False
